Note: I used a reference to perform LDA for this homework assignment. I also had completed the LDA portion before I was informed that we should not be doing LDA. I also consulted a classmate, Sali, for help on the homework.
https://yanlinc.medium.com/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6

In [2]:
import numpy as np
import re, nltk, gensim
import pandas as pd

## load dataset 
data = pd.read_csv(r'/Users/nikkigerjarusak/desktop/NLP/homework2/hw2.csv')
data.head(5)

,body
0,WASHINGTON The European Space Agency will esta...
1,Bucks County is rich in history exciting enter...
2,Steelhead Fishing Pulaski NY on the Salmon Riv...
3,Since the beginning of time exploring the Univ...
4,231 519 7348 Steelhead fishing is a popular s...


### 1.  Determine how many different topics, and what they are, exist in an aggregation of ALL corpuses together (50 points)

In [3]:
## convert to list
df = data.body.values.tolist()

In [4]:
## tokenization
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
data_words = list(sent_to_words(df))

print(data_words[:1])

[['washington', 'the', 'european', 'space', 'agency', 'will', 'establish', 'committee', 'with', 'representatives', 'from', 'both', 'inside', 'and', 'outside', 'the', 'space', 'industry', 'to', 'develop', 'options', 'for', 'european', 'human', 'space', 'exploration', 'program', 'the', 'creation', 'of', 'what', 'esa', 'called', 'high', 'level', 'advisory', 'group', 'was', 'one', 'of', 'the', 'major', 'outcomes', 'of', 'one', 'day', 'space', 'summit', 'held', 'in', 'toulouse', 'france', 'feb', 'that', 'brought', 'together', 'representatives', 'of', 'member', 'states', 'of', 'both', 'esa', 'and', 'the', 'european', 'union', 'to', 'discuss', 'future', 'european', 'space', 'initiatives', 'josef', 'aschbacher', 'director', 'general', 'of', 'esa', 'said', 'the', 'proposal', 'for', 'the', 'advisory', 'group', 'came', 'from', 'french', 'president', 'emmanuel', 'macron', 'we', 'got', 'very', 'clear', 'message', 'from', 'president', 'macron', 'that', 'such', 'group', 'is', 'needed', 'he', 'has', '

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [6]:
global dictionary
dictionary = nltk.corpus.words.words("en")
dictionary = [word.lower() for word in dictionary]

In [7]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [8]:
## lemmatization keeping adj, noun, verb, adverb
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN", "VERB"]) ## select noun and verb
print(data_lemmatized[:2])

['establish committee representative space industry develop option space exploration program creation call level group outcome day space summit hold bring representative member state discuss space initiative say proposal group come get message group need ask put group say press conference end summit speech day macron mention desire study option program summer want come target ambition space travel say interpreter need know priority datum back prepare choice go take meeting option say include mission end decade style project ambition say issue work decade say draft mandate group present member march meeting goal group start work committee prepare report time meeting report spring group comprise space expert say like look aspect society point view point view point view mean include people artist philosopher group look aspect exploration science technology preside summit meeting say group help establish strategy space exploration define strategy exploration say interpreter press conferenc

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(analyzer='word',
                             min_df=10, 
                             stop_words='english',
                             lowercase=True,
                             token_pattern='[a-zA-Z0-9]{3,}')

In [10]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [11]:
## build LDA model
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=20,
                                      max_iter=10,
                                      learning_method='online',
                                      random_state=123,
                                      batch_size=128,
                                      evaluate_every = -1,
                                      n_jobs = -1)
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=123)


In [12]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior = None,
                          evaluate_every = -1, learning_decay = 0.7,
                          learning_method = "online", learning_offset = 10.0,
                          max_doc_update_iter = 100, max_iter = 10, mean_change_tol = 0.001,
                          n_components = 10, n_jobs = -1, perp_tol = 0.1,
                          random_state = 123, topic_word_prior = None,
                          total_samples = 1000000.0, verbose = 0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=123)

In [13]:
from pprint import pprint
print("Log Likelihood: ", lda_model.score(data_vectorized)) ## higher the better
print("Perplexity: ", lda_model.perplexity(data_vectorized)) ## lower the better

## see model parameters
pprint(lda_model.get_params())

Log Likelihood:  -173480.7881187826
Perplexity:  317.11735113887187
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 123,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [14]:
from sklearn.model_selection import GridSearchCV
search_params = {"n_components": [10, 15, 20, 25, 30], "learning_decay": [.5, .7, .9]}


lda = LatentDirichletAllocation(max_iter = 5, learning_method = "online", 
                                learning_offset = 50.,random_state = 123)
## initite grid search
model = GridSearchCV(lda, param_grid=search_params)
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score="raise",
       estimator=LatentDirichletAllocation(batch_size = 128, doc_topic_prior = None,
             evaluate_every = -1, learning_decay = 0.7, learning_method = None,
             learning_offset = 10.0, max_doc_update_iter = 100, max_iter = 10,
             mean_change_tol = 0.001, n_components = 10, n_jobs = 1, perp_tol = 0.1, random_state = None,
             topic_word_prior = None, total_samples = 1000000.0, verbose = 0), n_jobs = 1, 
             param_grid = {"n_topics": [10, 15, 20, 25, 30], "learning_decay": [0.5, 0.7, 0.9]},
                           pre_dispatch = "2*n_jobs", refit=True, return_train_score="warn",
                           scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_topics': [10, 15, 20, 25, 30]},
             return_train_score='warn')

In [15]:
## best model
best_lda_model = model.best_estimator_

## model parameters
print("Best Model's Params: ", model.best_params_)

## log likelihood score
print("Best Log Likelihood Score: ", model.best_score_)

## perplexity 
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -40452.52286414245
Model Perplexity:  281.34507762755214


In [16]:
## create topic matrix
lda_output = best_lda_model.transform(data_vectorized)

## column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

## index names
docnames = ["Doc" + str(i) for i in range(len(data))]

data_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# get dominant topics
dominant_topic = np.argmax(data_document_topic.values, axis=1)
data_document_topic["dominant_topic"] = dominant_topic


In [17]:
data_document_topics = data_document_topic.head(15)
data_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8
Doc1,0.94,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0
Doc2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,4
Doc3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8
Doc4,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,4
Doc5,0.00,0.00,0.00,0.00,0.98,0.00,0.00,0.00,0.00,0.00,4
Doc6,0.71,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
Doc7,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
Doc8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8
Doc9,0.00,0.00,0.00,0.00,0.00,0.00,0.54,0.00,0.46,0.00,6


In [18]:
## topic matrix
data_topic_keywords = pd.DataFrame(best_lda_model.components_)
data_topic_keywords.columns = vectorizer.get_feature_names()
data_topic_keywords.index = topicnames

## view
data_topic_keywords.head()

,ability,abundance,access,accord,achieve,acre,action,activity,add,addition,...,wildlife,win,wind,winter,work,world,write,year,yellow,zone
Topic0,2.424234,3.504185,4.250168,14.554404,1.253739,14.199098,2.410140,6.278265,7.534372,1.227155,...,3.737884,3.479836,7.935273,13.983473,5.148095,10.504695,3.028305,47.268234,13.821573,3.353966
Topic1,1.699481,0.653881,1.413649,1.899005,1.368170,2.167230,1.067145,0.968104,1.397190,1.157308,...,1.085638,0.737426,0.965672,2.030767,2.176655,4.916617,0.763300,10.461317,0.998569,0.847728
Topic2,0.713470,1.090790,1.599110,0.566474,1.095286,0.521503,2.029519,0.877116,1.274699,0.829301,...,1.136841,1.859733,0.673462,9.599635,2.864779,1.702613,0.666437,6.951790,0.502727,0.566544
Topic3,0.753926,0.590945,0.570307,0.895452,0.850750,0.639680,0.621102,0.693264,0.600041,0.554351,...,0.643735,0.653135,0.728011,0.915441,1.070981,0.884829,0.723290,3.871515,0.842507,0.877031
Topic4,2.645929,1.028718,8.183882,2.220846,5.457207,0.568715,6.905824,1.881073,7.731384,3.346263,...,7.755903,2.795587,2.399993,30.569018,18.531008,7.077339,3.100619,23.789356,0.570708,4.713434


In [19]:
# show top 'n' keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)

# topic keywords data frame
data_topic_keywords = pd.DataFrame(topic_keywords)
data_topic_keywords.columns = ["Word" + str(i) for i in range(data_topic_keywords.shape[1])]
data_topic_keywords.index = ["Topic" + str(i) for i in range(data_topic_keywords.shape[0])]
data_topic_keywords

,Word0,Word1,Word2,Word3,Word4,Word5,Word6,Word7,Word8,Word9,Word10,Word11,Word12,Word13,Word14
Topic0,fall,color,foliage,tree,leave,peak,park,view,leaf,forest,season,change,maple,mountain,autumn
Topic1,space,fall,view,earth,spacecraft,park,astronaut,year,tree,say,foliage,change,orbit,moon,time
Topic2,email,fish,river,steelhead,fishing,follow,post,fly,angler,winter,water,time,trout,use,run
Topic3,space,color,fall,year,tree,salmon,foliage,forest,peak,fish,fishing,launch,moon,fly,maple
Topic4,fishing,fish,steelhead,river,run,salmon,water,fly,rod,angler,catch,trip,guide,line,use
Topic5,read,story,space,mission,fish,river,steelhead,earth,time,make,fishing,exploration,planet,asteroid,orbit
Topic6,space,mission,launch,earth,planet,spacecraft,year,orbit,moon,asteroid,rover,mar,study,date,astronaut
Topic7,space,color,peak,tree,company,foliage,leave,park,autumn,say,change,research,leaf,chlorophyll,spacecraft
Topic8,space,exploration,earth,mission,moon,spacecraft,use,launch,orbit,planet,astronaut,satellite,make,science,program
Topic9,photo,website,space,use,exploration,fish,fishing,steelhead,government,salmon,foliage,run,fall,share,design


In [20]:
Topics = ["Fall","Space Exploration","Fishing","Fall","Fishing",
          "Space Exploration", "Space Exploration", "Fall", "Space Exploration", "Fall"]
data_topic_keywords["Topics"] = Topics
data_topic_keywords

,Word0,Word1,Word2,Word3,Word4,Word5,Word6,Word7,Word8,Word9,Word10,Word11,Word12,Word13,Word14,Topics
Topic0,fall,color,foliage,tree,leave,peak,park,view,leaf,forest,season,change,maple,mountain,autumn,Fall
Topic1,space,fall,view,earth,spacecraft,park,astronaut,year,tree,say,foliage,change,orbit,moon,time,Space Exploration
Topic2,email,fish,river,steelhead,fishing,follow,post,fly,angler,winter,water,time,trout,use,run,Fishing
Topic3,space,color,fall,year,tree,salmon,foliage,forest,peak,fish,fishing,launch,moon,fly,maple,Fall
Topic4,fishing,fish,steelhead,river,run,salmon,water,fly,rod,angler,catch,trip,guide,line,use,Fishing
Topic5,read,story,space,mission,fish,river,steelhead,earth,time,make,fishing,exploration,planet,asteroid,orbit,Space Exploration
Topic6,space,mission,launch,earth,planet,spacecraft,year,orbit,moon,asteroid,rover,mar,study,date,astronaut,Space Exploration
Topic7,space,color,peak,tree,company,foliage,leave,park,autumn,say,change,research,leaf,chlorophyll,spacecraft,Fall
Topic8,space,exploration,earth,mission,moon,spacecraft,use,launch,orbit,planet,astronaut,satellite,make,science,program,Space Exploration
Topic9,photo,website,space,use,exploration,fish,fishing,steelhead,government,salmon,foliage,run,fall,share,design,Fall


There are 3 different topics contained in the document. The topics are Fall, Space Exploration, and Fishing. 

### 2.  Once you get a the list of topics, assign each corpus to a topic through an automated tagging process (assign each corpus to a topic) of your choice (50 points)

In [21]:
## create function assign topics to each corpus
def topics(t):
    if t == 0:
        return "Fall"
    elif t == 1:
        return "Space Exploration"
    elif t == 2:
        return "Fishing"
    elif t == 3:
        return "Fall"
    elif t == 4:
        return "Fishing"
    elif t == 5:
        return "Space Exploration"
    elif t == 6:
        return "Space Exploration"
    elif t == 7:
        return "Fall"
    elif t == 8:
        return "Space Exploration"
    else:
        return "Fall"

In [23]:
## apply function
data_document_topic['Topic Assignment'] = data_document_topic['dominant_topic'].apply(topics)

In [24]:
data_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic,Topic Assignment
Doc0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8,Space Exploration
Doc1,0.94,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0,Fall
Doc2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,4,Fishing
Doc3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.99,0.00,8,Space Exploration
Doc4,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,4,Fishing
...,...,...,...,...,...,...,...,...,...,...,...,...
Doc119,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,Fall
Doc120,0.00,0.00,0.00,0.00,0.99,0.00,0.00,0.00,0.00,0.00,4,Fishing
Doc121,0.09,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.85,0.01,8,Space Exploration
Doc122,0.00,0.00,0.00,0.00,0.00,0.00,0.77,0.00,0.23,0.00,6,Space Exploration
